In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
time_origin = datetime.datetime(2010,1,1,0,0,0)
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')
import plot_water_table2D
sys.path.append(os.getcwd())
import SiteC_2012_2014WL 

In [3]:
reload(SiteC_2012_2014WL)

data2012 = SiteC_2012_2014WL.getData_2012()

In [4]:
reload(SiteC_2012_2014WL)
ObservedData2013 = dict()
data2013 = SiteC_2012_2014WL.getData_2013()

In [5]:
reload(SiteC_2012_2014WL)
ObservedData2014 = dict()
data2014 = SiteC_2012_2014WL.getData_2014()

In [6]:
reload(SiteC_2012_2014WL)
dataPrecip = SiteC_2012_2014WL.getDataRain()

[889.75       889.76041667 889.77083333]


In [7]:
ObservedData2012 = dict()
ObservedData2013 = dict()
ObservedData2014 = dict()

for key in data2013.keys():
    ObservedData2012[key] = data2012[key]
    ObservedData2013[key] = data2013[key]
    ObservedData2014[key] = data2014[key]

In [8]:
simulation = 'SiteC_1R1'

reload(plot_water_table2D)
#path = "/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC_New/"
path = "/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC_March4/"

Time, WaterLevelCenter, ThawDepthC, Z_c = plot_water_table2D.water_table2D(path + '%s/'%simulation)#,x_pos= 36)
Time, WaterLevelTrough, ThawDepthT, Z_t = plot_water_table2D.water_table2D(path + '%s/'%simulation, location='trough')#,x_pos=75)

Location:  center 45
('Datum (surface elevation)', array([4.78019978]))
Location:  trough 1
('Datum (surface elevation)', array([4.71264008]))


In [11]:
# correcting the elevation of the center
#Z_c = 4.78
#Z_t = 4.71
def OffsetData():
    for key in ObservedData2012.keys():
        if 'water_level_C40' in key:
            ObservedData2012[key + '_offset'] = np.array([d - 4.91 + Z_c for d in ObservedData2012[key]])
    for key in ObservedData2012.keys():
        if 'water_level_C39' in key:
            ObservedData2012[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2012[key]])
    for key in ObservedData2013.keys():
        if 'water_level_C39' in key:
            ObservedData2013[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2013[key]])
    for key in ObservedData2014.keys():
        if 'water_level_C39' in key:
            ObservedData2014[key + '_offset'] = np.array([d - 4.81 + Z_c for d in ObservedData2014[key]])

In [12]:
DataSim =dict()
time = [3*365*0 + t*365.25 for t in Time]

DataSim['time'] = time
DataSim['WL_center'] = WaterLevelCenter
DataSim['WL_trough'] = WaterLevelTrough
DataSim['TD_center'] = [t + Z_c for t in ThawDepthC]
DataSim['TD_trough'] = [t + Z_t for t in ThawDepthT]

Data_C = []
Data_T = []
Time_C = []
Time_T = []
for i,d in enumerate(DataSim['WL_center']):
    if d != 0:
        Data_C.append(d + Z_c)
        Time_C.append(DataSim['time'][i])
for i,d in enumerate(DataSim['WL_trough']):
    if d != 0:
        Data_T.append(d + Z_t)
        Time_T.append(DataSim['time'][i])

DataSim['water_level_center'] = np.array(Data_C)
DataSim['water_level_trough'] = np.array(Data_T)
DataSim['time_center'] = np.array(Time_C)
DataSim['time_trough'] = np.array(Time_T)

OffsetData()

In [13]:
#--- Remove the assigned 100 to the missing data points

alpha1, alpha2 = 1, 0
LC39 = len(ObservedData2012['water_level_C39_offset'])
mask = np.ones(LC39, dtype=bool)

for i,d1 in enumerate(ObservedData2012['water_level_C39_offset']):
    if d1 > 10:
        mask[i] = False

ObservedData2012['water_level_C39_mask'] = ObservedData2012['water_level_C39_offset'][mask]
ObservedData2012['time_C39_mask'] = ObservedData2012['time_C39'][mask]

#-----------------------------------------------------------------


# Average the data at the two center wells
assert len(ObservedData2012['water_level_C39']) == len(ObservedData2012['water_level_C40_offset'])
data = []
index = np.where(ObservedData2012['water_level_C39_offset'] > 10)[0]

mid_value = 0.5*(ObservedData2012['water_level_C39_offset'][index[0]-1] + ObservedData2012['water_level_C39_offset'][index[-1]+1])

for d1,d2 in zip(ObservedData2012['water_level_C39_offset'], ObservedData2012['water_level_C40_offset']):
    sum = 0
    if d1 < 10:
        sum = 0.5*(d1+d2)
    else:
        sum = 0.5*(d2 + mid_value)
    data.append(sum)
ObservedData2012['water_level_Center_offset'] = np.concatenate(data)

print len(ObservedData2013['water_level_C39_offset']), len(ObservedData2013['water_level_C40'])

data = []
c = 0
for i in range(len(ObservedData2013['water_level_C39_offset'])):
    if i <7:
        data.append(ObservedData2013['water_level_C39_offset'][i])
    elif (i < 113):
        sum = alpha1*(ObservedData2013['water_level_C39_offset'][i] + alpha2*ObservedData2013['water_level_C40'][i-7])    
        data.append(sum)
    else:
        data.append(ObservedData2013['water_level_C39_offset'][i])


ObservedData2013['water_level_Center'] = np.concatenate(data)

#--- 2014
print len(ObservedData2014['water_level_C39']), len(ObservedData2014['water_level_C40'])
data = []
for i in range(len(ObservedData2014['water_level_C39_offset'])):
        sum = alpha1*(ObservedData2014['water_level_C39_offset'][i] + alpha2*ObservedData2014['water_level_C40'][i])    
        data.append(sum)

ObservedData2014['water_level_Center'] = np.concatenate(data)

114 106
85 85


In [21]:
def PlotWithCenterAverage():
    outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/March4/'#/SiteC_3Runs/'
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()

    ax = plt.subplot(321)
    plt.fill_between(ObservedData2012['time_C37'], ObservedData2012['water_level_C37']-0.04, 
                     ObservedData2012['water_level_C37']+ 0.04, color='c')
    plt.plot(ObservedData2012['time_C37'], ObservedData2012['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')
    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(882, 4.4, 'Trough, 2012')
    plt.ylabel('Water level')

    #ax.set_xticks(np.arange(880, 1001, step=20))
    #ax.set_xticklabels(timeadj2012)
    ax.set_xlim(880, 1000)
    #plt.margins(0.2)
    plt.ylim(4.2, 5.2)


    ax = plt.subplot(322)
    print '2012: ', ObservedData2012['water_level_Center_offset'].shape
    plt.fill_between(ObservedData2012['time_C39'], ObservedData2012['water_level_Center_offset']-0.04, 
                     ObservedData2012['water_level_Center_offset']+ 0.04, color='c')
    plt.plot(ObservedData2012['time_C40'], ObservedData2012['water_level_Center_offset'],'r')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')
    
    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.text(882, 4.4, 'Center, 2012')
    plt.xlim(880, 1000)
    plt.ylim(4.2, 5.2)
    """
    ax1 = ax.twinx()
    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])
    """
    #---------------------------------------------------------
    # ------------ 2013 --------------------------------------
    ax = plt.subplot(323)
    plt.fill_between(ObservedData2013['time_C37'], ObservedData2013['water_level_C37']-0.04, 
                     ObservedData2013['water_level_C37']+ 0.04, color='c')
    plt.plot(ObservedData2013['time_C37'], ObservedData2013['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')


    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1238, 4.4, 'Trough, 2013')
    #plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)


    ax = plt.subplot(324)
    
    plt.fill_between(ObservedData2013['time_C39'], ObservedData2013['water_level_Center']-0.04, 
                     ObservedData2013['water_level_Center']+ 0.04, color='c')
    plt.plot(ObservedData2013['time_C39'], ObservedData2013['water_level_Center'],'r')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')


    plt.text(1238, 4.4, 'Center, 2013')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)
    """
    ax1 = ax.twinx()

    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])
    """

    #---------------- 2014 -----------------------------
    # --------------------------------------------------
    ax = plt.subplot(325)
    plt.fill_between(ObservedData2014['time_C37'], ObservedData2014['water_level_C37']-0.04, 
                     ObservedData2014['water_level_C37']+ 0.04, color='c')
    plt.plot(ObservedData2014['time_C37'], ObservedData2014['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.text(1622, 4.4, 'Trough, 2014')
    plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)
    #plt.legend()

    ax = plt.subplot(326)
    plt.fill_between(ObservedData2014['time_C39'], ObservedData2014['water_level_Center']-0.04, 
                     ObservedData2014['water_level_Center']+ 0.04, color='c')
    plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_Center'],'r', label='Observed')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4',label='Thaw depth')
    
    plt.text(1622, 4.4, 'Center, 2014')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.legend()
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)
    """
    ax1 = ax.twinx()
    #ax1.plot(DataSim['time'],DataSim['Rain_precip'],'y')
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    ax1.set_ylim([-17.0, 10])
    ax1.set_yticks([0, 5, 10])
    """
    ax.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))
    plt.tight_layout()
    plt.savefig(outfile+'%s-avg.pdf'%simulation, bbox_inches='tight', dpi=100)
    plt.show()




In [22]:
def PlotWithoutCenterAverage():
    outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/Feb20/'
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
   
    ax = plt.subplot(321)
    plt.plot(ObservedData2012['time_C37'], ObservedData2012['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')
    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.text(882, 4.6, 'Trough, 2012')
    plt.ylabel('Water level')

    ax.set_xlim(880, 1000)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(322)
    plt.plot(ObservedData2012['time_C40'], ObservedData2012['water_level_C40_offset'],'r')
    plt.plot(ObservedData2012['time_C39_mask'], ObservedData2012['water_level_C39_mask'],'g')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')

    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.text(882, 4.6, 'Center, 2012')
    plt.xlim(880, 1000)
    plt.ylim(4.3, 5.1)
    
    """
    ax1 = ax.twinx()
    ax1.plot(dataPrecip['time_2012'],dataPrecip['rain_2012'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')

    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """
    
    #---------------------------------------------------------
    # ------------ 2013 --------------------------------------
    ax = plt.subplot(323)
    plt.plot(ObservedData2013['time_C37'], ObservedData2013['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.text(1238, 4.6, 'Trough, 2013')
    #plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(324)
    
    #plt.plot(ObservedData2013['time_C40'], ObservedData2013['water_level_C40'],'r')
    plt.plot(ObservedData2013['time_C39'], ObservedData2013['water_level_C39_offset'],'g')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4')


    plt.text(1238, 4.6, 'Center, 2013')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    #plt.legend()
    plt.xlim(1236, 1362)
    plt.ylim(4.3, 5.1)
    #ax1 = ax.twinx()

    """
    ax1.plot(dataPrecip['time_2013'],dataPrecip['rain_2013'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')

    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """

    #---------------- 2014 -----------------------------
    # --------------------------------------------------
    ax = plt.subplot(325)
    plt.plot(ObservedData2014['time_C37'], ObservedData2014['water_level_C37'],'r')
    plt.plot(DataSim['time_trough'], DataSim['water_level_trough'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_trough'],'m',markersize='4')

    plt.axhline(y=Z_t,linestyle='--',color='g')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.text(1622, 4.6, 'Trough, 2014')
    plt.xlabel('Day')
    plt.ylabel('Water level')
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)

    ax = plt.subplot(326)

    #pondedepth40 = [x - 4.91 for x in ObservedData2014['water_level_C40']]
    #plt.plot(ObservedData2014['time_C40'], ObservedData2014['water_level_C40'],'r',label='Observed (C40)')
    plt.plot(ObservedData2014['time_C39'], ObservedData2014['water_level_C39_offset'],'g',label='Observed (C39)')
    plt.plot(DataSim['time_center'], DataSim['water_level_center'],'k*',markersize='4',label='Simulated')
    plt.plot(DataSim['time'], DataSim['TD_center'],'m',markersize='4',label='Thaw depth')

    plt.text(1622, 4.6, 'Center, 2014')
    plt.axhline(y=Z_c,linestyle='--',color='g')
    plt.legend()
    plt.xlim(1620, 1710)
    plt.ylim(4.3, 5.1)
    
    """
    ax1 = ax.twinx()
    ax1.plot(dataPrecip['time_2014'],dataPrecip['rain_2014'],'c', label='Observed rain')
    ax1.plot(DataSim['Time_forcing'],DataSim['Rain_forcing'],'g.', label='Forcing data (rain)')
    
    ax1.set_ylabel('Precipitation Rain [mm]',color='y')
    ax1.tick_params('y', colors='y')
    #ax1.set_ylim([-17.0, 10])
    #ax1.set_yticks([0, 5, 10])
    """
    ax.legend(loc='upper right', fontsize=7, ncol=4, bbox_to_anchor=(0.0,-0.32, .1, .1))
    #ax1.legend(loc='upper right', fontsize=7, ncol=2, bbox_to_anchor=(0.8,- 0.32, .1, .1))
    plt.tight_layout()
    plt.savefig(outfile+'%s.pdf'%simulation, bbox_inches='tight', dpi=100)
    plt.show()

In [23]:
PlotWithCenterAverage()
#PlotWithoutCenterAverage()

2012:  (105,)
